In [1]:
import numpy as np
path = '2M0030/2M0030_'
rr = np.loadtxt(path.split('/')[0]+'/initialguess')
xca, yca, xcb, ycb = rr[0], rr[1], rr[2], rr[3]
print(rr[0], rr[1], rr[2], rr[3])

from cliotools.bditools import findstars_in_dataset
#path = 'UCAC2/UCAC2_'

rr = np.loadtxt(path.split('/')[0]+'/initialguess')
#xca, yca = 466, 183 
#xcb, ycb = 812, 116

findstars_in_dataset(path, rr[0], rr[1], corrboxsizex = 40, corrboxsizey = 40,\
                     boxsizex = 40, boxsizey = 40, skip_list = False, \
                     append_file = False, threshold = 1e3, radius = 40)

333.0 122.0 291.0 398.0
100% (120 of 120): |####################|  Done...
Done
